In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os

class SimpleSeismicNoiseClassifier:
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=50, random_state=42)
        self.scaler = StandardScaler()
        self.is_trained = False
    
    def extract_simple_features(self, image):
        """
        Extract simple but effective features for seismic noise detection
        """
        # Convert to grayscale
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Normalize
        gray = gray.astype(np.float32) / 255.0
        
        features = []
        
        # 1. Basic statistical features
        features.extend([
            np.mean(gray),           # Average brightness
            np.std(gray),            # Contrast measure
            np.var(gray),            # Variance
            np.min(gray),            # Minimum value
            np.max(gray),            # Maximum value
        ])
        
        # 2. Texture features using gradients
        grad_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
        gradient_mag = np.sqrt(grad_x**2 + grad_y**2)
        
        features.extend([
            np.mean(gradient_mag),   # Edge strength
            np.std(gradient_mag),    # Edge variation
        ])
        
        # 3. Frequency domain features (simple)
        fft = np.fft.fft2(gray)
        fft_shift = np.fft.fftshift(fft)
        magnitude = np.abs(fft_shift)
        
        features.extend([
            np.mean(magnitude),      # Frequency content
            np.std(magnitude),       # Frequency variation
        ])
        
        # 4. Noise-specific features
        # High frequency noise detection
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        noise = gray - blur
        
        features.extend([
            np.std(noise),           # High frequency noise level
            np.mean(np.abs(noise)),  # Absolute noise level
        ])
        
        return np.array(features)
    
    def create_training_data(self, clean_images, noisy_images):
        """
        Prepare training data from clean and noisy images
        """
        X = []
        y = []
        
        print("Processing clean images...")
        for img in clean_images:
            features = self.extract_simple_features(img)
            X.append(features)
            y.append(0)  # 0 = clean/no noise
        
        print("Processing noisy images...")
        for img in noisy_images:
            features = self.extract_simple_features(img)
            X.append(features)
            y.append(1)  # 1 = noisy
        
        return np.array(X), np.array(y)
    
    def train(self, clean_images, noisy_images):
        """
        Train the classifier
        """
        print("Extracting features...")
        X, y = self.create_training_data(clean_images, noisy_images)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        print("Training model...")
        self.model.fit(X_train_scaled, y_train)
        
        # Test model
        y_pred = self.model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"\nModel trained successfully!")
        print(f"Accuracy: {accuracy:.2%}")
        print("\nDetailed Report:")
        print(classification_report(y_test, y_pred, 
                                  target_names=['Clean', 'Noisy']))
        
        self.is_trained = True
        return accuracy
    
    def predict(self, image, show_result=True):
        """
        Predict if an image contains noise
        """
        if not self.is_trained:
            print("Model not trained yet!")
            return None
        
        features = self.extract_simple_features(image).reshape(1, -1)
        features_scaled = self.scaler.transform(features)
        
        prediction = self.model.predict(features_scaled)[0]
        probability = self.model.predict_proba(features_scaled)[0]
        
        result = {
            'has_noise': bool(prediction),
            'confidence': float(max(probability)),
            'clean_prob': float(probability[0]),
            'noisy_prob': float(probability[1])
        }
        
        if show_result:
            plt.figure(figsize=(12, 4))
            
            # Show image
            plt.subplot(1, 2, 1)
            if len(image.shape) == 3:
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            else:
                plt.imshow(image, cmap='seismic')
            plt.title('Input Image')
            plt.axis('off')
            
            # Show prediction
            plt.subplot(1, 2, 2)
            labels = ['Clean', 'Noisy']
            colors = ['green', 'red']
            bars = plt.bar(labels, probability, color=colors, alpha=0.7)
            plt.title(f'Prediction: {"NOISY" if prediction else "CLEAN"}\n'
                     f'Confidence: {max(probability):.1%}')
            plt.ylabel('Probability')
            plt.ylim(0, 1)
            
            # Add value labels on bars
            for bar, prob in zip(bars, probability):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                        f'{prob:.1%}', ha='center', va='bottom')
            
            plt.tight_layout()
            plt.show()
        
        return result



def demonstrate_classifier():
    print("=== SONATRACH Seismic Noise Detection Demo ===\n")
    
    # Initialize classifier
    classifier = SimpleSeismicNoiseClassifier()
    
    print("Creating synthetic demo data...")
    
    # Generate synthetic clean images (smooth patterns)
    clean_images = []
    for i in range(20):
        img = np.zeros((100, 100), dtype=np.uint8)
        for j in range(10):
            y = int(10 + j*8 + 5*np.sin(i*0.5))
            img[y:y+2, :] = 200
        clean_images.append(img)
    
    # Generate synthetic noisy images (irregular patterns)
    noisy_images = []
    for i in range(20):
        img = np.zeros((100, 100), dtype=np.uint8)
        for j in range(10):
            y = int(10 + j*8 + 5*np.sin(i*0.5))
            img[y:y+2, :] = 200
        # Add random noise (simulating interference)
        noise = np.random.randint(0, 100, (100, 100))
        img = cv2.addWeighted(img.astype(np.uint8), 0.7, noise.astype(np.uint8), 0.3, 0)
        noisy_images.append(img)
    
    # Train the model
    accuracy = classifier.train(clean_images, noisy_images)
    
    # Test on new image
    test_clean = clean_images[0]
    test_noisy = noisy_images[0]
    
    print("\n=== Testing on Clean Image ===")
    result_clean = classifier.predict(test_clean)
    
    print("\n=== Testing on Noisy Image ===")
    result_noisy = classifier.predict(test_noisy)
    
    return classifier

def load_images_from_folder(folder_path):
    """
    Load all images from a folder
    """
    images = []
    filenames = []
    
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return images, filenames
    
    print(f"Loading images from: {folder_path}")
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.tif')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
                filenames.append(filename)
                print(f"  Loaded: {filename}")
            else:
                print(f"  Failed to load: {filename}")
    
    print(f"Total images loaded: {len(images)}")
    return images, filenames

def test_on_your_images(classifier, test_folder_path, save_results=True):
    """
    Test the trained classifier on your test images
    """
    if not classifier.is_trained:
        print("Error: Classifier is not trained yet!")
        return None
    
    # Load test images
    test_images, test_filenames = load_images_from_folder(test_folder_path)
    
    if len(test_images) == 0:
        print("No test images found!")
        return None
    
    print(f"\n=== Testing on {len(test_images)} images ===")
    
    results = []
    
    for i, (img, filename) in enumerate(zip(test_images, test_filenames)):
        print(f"\nProcessing {i+1}/{len(test_images)}: {filename}")
        
        # Get prediction (with visualization)
        result = classifier.predict(img, show_result=True)
        
        # Store result
        result_dict = {
            'filename': filename,
            'has_noise': result['has_noise'],
            'confidence': result['confidence'],
            'clean_probability': result['clean_prob'],
            'noisy_probability': result['noisy_prob']
        }
        results.append(result_dict)
        
        print(f"  Result: {'NOISY' if result['has_noise'] else 'CLEAN'}")
        print(f"  Confidence: {result['confidence']:.1%}")
        print(f"  Clean prob: {result['clean_prob']:.1%}")
        print(f"  Noisy prob: {result['noisy_prob']:.1%}")
    
    # Save results to CSV
    if save_results:
        results_df = pd.DataFrame(results)
        results_file = 'test_results.csv'
        results_df.to_csv(results_file, index=False)
        print(f"\n=== Results saved to: {results_file} ===")
        
        # Print summary
        noisy_count = sum(1 for r in results if r['has_noise'])
        clean_count = len(results) - noisy_count
        
        print(f"\nSUMMARY:")
        print(f"Total images tested: {len(results)}")
        print(f"Predicted as CLEAN: {clean_count}")
        print(f"Predicted as NOISY: {noisy_count}")
        print(f"Average confidence: {np.mean([r['confidence'] for r in results]):.1%}")
        
        return results_df
    
    return results

def process_your_seismic_data(training_folder_path, test_folder_path=None):
    """
    Complete workflow for processing your actual seismic data
    
    Parameters:
    - training_folder_path: Path to folder containing 'clean' and 'noisy' subfolders
    - test_folder_path: Path to folder containing test images (optional)
    """
    classifier = SimpleSeismicNoiseClassifier()
    
    # Step 1: Load training images
    clean_folder = os.path.join(training_folder_path, 'clean')
    noisy_folder = os.path.join(training_folder_path, 'noisy')
    
    print("=== LOADING TRAINING DATA ===")
    clean_images, clean_filenames = load_images_from_folder(clean_folder)
    noisy_images, noisy_filenames = load_images_from_folder(noisy_folder)
    
    if len(clean_images) == 0 or len(noisy_images) == 0:
        print("Error: Need both clean and noisy images for training!")
        print(f"Clean images: {len(clean_images)}")
        print(f"Noisy images: {len(noisy_images)}")
        return None
    
    print(f"\nTraining data summary:")
    print(f"Clean images: {len(clean_images)}")
    print(f"Noisy images: {len(noisy_images)}")
    
    # Step 2: Train the model
    print(f"\n=== TRAINING MODEL ===")
    accuracy = classifier.train(clean_images, noisy_images)
    
    # Step 3: Test on your test images
    if test_folder_path:
        print(f"\n=== TESTING ON YOUR IMAGES ===")
        results = test_on_your_images(classifier, test_folder_path)
        return classifier, results
    else:
        print("\nNo test folder specified. Model is trained and ready for testing.")
        return classifier, None

def test_single_image(classifier, image_path):
    """
    Test a single image file
    """
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        return None
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image: {image_path}")
        return None
    
    print(f"Testing image: {os.path.basename(image_path)}")
    result = classifier.predict(img, show_result=True)
    
    return result

# Main execution
if __name__ == "__main__":
    # After training
    classifier, _ = process_your_seismic_data('train')

# Test all images in your test folder
results = test_on_your_images(classifier, 'test')